# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking current working directory
print(os.getcwd())

# Get current folder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []
# Get each file path directly inside this directory, not subdirectories
for item in os.listdir(filepath):
    full_path = os.path.join(filepath, item)
    if os.path.isfile(full_path):
        file_path_list.append(full_path)
        
print(len(file_path_list))

/home/workspace
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# Reading csv file
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)        
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line)

# Total number of rows
print(len(full_data_rows_list))

# Creating a smaller event data csv file that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## Query 1 Description: In this query, I used sessionId as the partition key and itemInSession as my clustering key. Each partition is uniquely identified by sessionId while itemInSession was used to uniquely identify the rows within a partition to sort the data by the item in session number.
query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# Ingest CSV data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "UPDATE song_info_by_session SET artist = %s, song = %s, length = %s WHERE sessionId = %s AND itemInSession = %s"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Sample SELECT statement to verify the data was entered into the table correctly
query = "SELECT artist, song, length FROM song_info_by_session WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## Query 2 Description: In this query, I used userId and sessionId as the composite partition key, with itemInSession as my clustering key. Each partition is uniquely identified by the combination of userId and sessionId, though neither uniquely identifies a partition on its own (which is why I used both together). itemInSession was used to uniquely identify the rows within a partition to sort the data by the item in session number.

query = "CREATE TABLE IF NOT EXISTS artist_song_by_user "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, username text,  PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    

In [12]:
# Ingest CSV data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "UPDATE artist_song_by_user SET artist = %s, song = %s, username = %s WHERE userId = %s AND sessionId = %s AND itemInSession = %s"
        # I combine the firstName and lastName fields with a space and store them as a single 'username' value in the database
        # This makes sense because the firstName and lastName are never used separately in the query.
        session.execute(query, (line[0], line[9], line[1] + " " + line[4], int(line[10]), int(line[8]), int(line[3])))

In [13]:
## Sample SELECT statement to verify the data was entered into the table correctly
query = "SELECT artist, song, username FROM artist_song_by_user WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.username)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## Query 3 Description: In this query, I used the song as the partition key and the userId as my clustering key. Each partition is uniquely identified by the song, which is our query conditional. And by clustering by the userId, we can identify unique entries for a particular song's listener (i.e. userId). Since the username isn't unique, we use the userId to make the entry unique, in case two users listen to the same song and have the same name.

query = "CREATE TABLE IF NOT EXISTS users_by_song "
query = query + "(song text, userId int, username text,  PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
# Ingest CSV data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, userId, username) VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1] + " " + line[4]))

In [20]:
## Sample SELECT statement to verify the data was entered into the table correctly
query = "SELECT song, username FROM users_by_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.song, row.username)

All Hands Against His Own Jacqueline Lynch
All Hands Against His Own Tegan Levine
All Hands Against His Own Sara Johnson


### Drop the tables before closing out the sessions

In [16]:
## Drop the tables before closing out the sessions
query = "DROP TABLE IF EXISTS song_info_by_session"
query1 = "DROP TABLE IF EXISTS artist_song_by_user"
query2 = "DROP TABLE IF EXISTS users_by_song"
try:
    rows = session.execute(query)
    rows = session.execute(query1)
    rows = session.execute(query2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()